In [1]:
#necessary libraries of pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, IntegerType

In [2]:
spark = SparkSession.builder.appName("demo").getOrCreate()


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/04/19 20:53:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/04/19 20:53:28 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
employees = [(1, "Scott", "Tiger", 1000.0, 10,
                      "united states", "+1 123 456 7890", "123 45 6789"
                     ),
                     (2, "Henry", "Ford", 1250.0, None,
                      "India", "+91 234 567 8901", "456 78 9123"
                     ),
                     (3, "Nick", "Junior", 750.0, '',
                      "united KINGDOM", "+44 111 111 1111", "222 33 4444"
                     ),
                     (4, "Bill", "Gomes", 1500.0, 10,
                      "AUSTRALIA", "+61 987 654 3210", "789 12 6118"
                     )
                ]

In [4]:
employeesDF = spark. \
    createDataFrame(employees,
                    schema="""employee_id INT, first_name STRING, 
                    last_name STRING, salary FLOAT, bonus STRING, nationality STRING,
                    phone_number STRING, ssn STRING"""
                   )

In [5]:
employeesDF.show()

+-----------+----------+---------+------+-----+--------------+----------------+-----------+
|employee_id|first_name|last_name|salary|bonus|   nationality|    phone_number|        ssn|
+-----------+----------+---------+------+-----+--------------+----------------+-----------+
|          1|     Scott|    Tiger|1000.0|   10| united states| +1 123 456 7890|123 45 6789|
|          2|     Henry|     Ford|1250.0| null|         India|+91 234 567 8901|456 78 9123|
|          3|      Nick|   Junior| 750.0|     |united KINGDOM|+44 111 111 1111|222 33 4444|
|          4|      Bill|    Gomes|1500.0|   10|     AUSTRALIA|+61 987 654 3210|789 12 6118|
+-----------+----------+---------+------+-----+--------------+----------------+-----------+




* Create a dataframe using list called as persons and categorize them based up on following rules.

|Age range|Category|
|---------|--------|
|0 to 2 Months|New Born|
|2+ Months to 12 Months|Infant|
|12+ Months to 48 Months|Toddler|
|48+ Months to 144 Months|Kids|
|144+ Months|Teenager or Adult|

In [7]:
persons = [
    (1, 1),
    (2, 13),
    (3, 18),
    (4, 60),
    (5, 120),
    (6, 0),
    (7, 12),
    (8, 160)
]

In [18]:
personsDF = spark.createDataFrame(persons).toDF("id",'age')

In [19]:
personsDF.printSchema()

root
 |-- id: long (nullable = true)
 |-- age: long (nullable = true)



In [20]:
personsDF.show()

+---+---+
| id|age|
+---+---+
|  1|  1|
|  2| 13|
|  3| 18|
|  4| 60|
|  5|120|
|  6|  0|
|  7| 12|
|  8|160|
+---+---+



In [21]:
from pyspark.sql.functions import expr

In [26]:
personsDF.withColumn("category",
expr("""
    CASE
    WHEN age BETWEEN 0 AND 2 THEN "nEW bORN"
    WHEN age >2 AND age <12 THEN "INFENT"
    ELSE "TEENAGER"
    END
""")).show()

+---+---+--------+
| id|age|category|
+---+---+--------+
|  1|  1|nEW bORN|
|  2| 13|TEENAGER|
|  3| 18|TEENAGER|
|  4| 60|TEENAGER|
|  5|120|TEENAGER|
|  6|  0|nEW bORN|
|  7| 12|TEENAGER|
|  8|160|TEENAGER|
+---+---+--------+



In [16]:
personsDF = spark.createDataFrame(persons, schema='id INT, age INT')


In [23]:
personsDF. \
    withColumn(
        'category',
        expr("""
            CASE
            WHEN age BETWEEN 0 AND 2 THEN 'New Born'
            WHEN age > 2 AND age <= 12 THEN 'Infant'
            WHEN age > 12 AND age <= 48 THEN 'Toddler'
            WHEN age > 48 AND age <= 144 THEN 'Kid'
            ELSE 'Teenager or Adult'
            END
        """)
    ). \
    show()

+---+---+-----------------+
| id|age|         category|
+---+---+-----------------+
|  1|  1|         New Born|
|  2| 13|          Toddler|
|  3| 18|          Toddler|
|  4| 60|              Kid|
|  5|120|              Kid|
|  6|  0|         New Born|
|  7| 12|           Infant|
|  8|160|Teenager or Adult|
+---+---+-----------------+



In [34]:
from pyspark.sql.functions import when,col

In [37]:
personsDF.printSchema()

root
 |-- id: long (nullable = true)
 |-- age: long (nullable = true)



In [46]:
personsDF.withColumn('category',
when(col('age').between(0,2),"new born").\
when((col('age')>2) & (col('age')<=12), "infant").\
when((col('age')>12) & (col('age')<=48), "toddler").\
when((col('age')>2) & (col('age')<12), "kid").\
otherwise("teenager")).show()

+---+---+--------+
| id|age|category|
+---+---+--------+
|  1|  1|new born|
|  2| 13| toddler|
|  3| 18| toddler|
|  4| 60|teenager|
|  5|120|teenager|
|  6|  0|new born|
|  7| 12|  infant|
|  8|160|teenager|
+---+---+--------+



In [41]:
personsDF.withColumn("category", when(col('age').between(0,2),'new born')).show()

+---+---+--------+
| id|age|category|
+---+---+--------+
|  1|  1|new born|
|  2| 13|    null|
|  3| 18|    null|
|  4| 60|    null|
|  5|120|    null|
|  6|  0|new born|
|  7| 12|    null|
|  8|160|    null|
+---+---+--------+



In [39]:
personsDF. \
    withColumn(
        'category',
        when(col('age').between(0, 2), 'New Born').
        when((col('age') > 2) & (col('age') <= 12), 'Infant').
        when((col('age') > 12) & (col('age') <= 48), 'Toddler').
        when((col('age') > 48) & (col('age') <= 144), 'Kid').
        otherwise('Teenager or Adult')
    ). \
    show()

+---+---+-----------------+
| id|age|         category|
+---+---+-----------------+
|  1|  1|         New Born|
|  2| 13|          Toddler|
|  3| 18|          Toddler|
|  4| 60|              Kid|
|  5|120|              Kid|
|  6|  0|         New Born|
|  7| 12|           Infant|
|  8|160|Teenager or Adult|
+---+---+-----------------+

